In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
import json
import os

options = webdriver.EdgeOptions()
options.binary_location = "/usr/bin/microsoft-edge"
options.add_experimental_option("debuggerAddress", "127.0.0.1:5555")

service = Service(EdgeChromiumDriverManager().install())

br = webdriver.Edge(options, service)

chunk_size = 400
actions = ActionChains(br)

with open("florida.txt", "r") as f:
    domains = f.read()

list_domain = [t_.strip() for t_ in domains.split("\n")]
len(list_domain)


def download(k):
    list_text = list_domain[k*chunk_size:(k+1)*chunk_size]
    input_box = br.find_element(By.ID, "recherche-domaine-input_multiple")
    button = br.find_element(By.ID, "trigger-search-multiple")
    input_box.clear()
    input_box.send_keys("\n".join(list_text))
    button.click()
    time.sleep(5)

    a = time.time()
    while True:
        try:
            export_button = br.find_element(By.ID, "exportResults")
            break
        except:
            time.sleep(1)
            if time.time() - a >= 30:
                return False
    
    while True:
        try:
            export_button.click()
            n = len(os.listdir("save_1/"))
            break
        except:
            time.sleep(1)
    
    a = time.time()
    while len(os.listdir("save_1/")) == n:
        time.sleep(1)
        if time.time() - a >= 30:
            return False
    
    return True


def run(start, end):
    for k in range(start, end):
        br.find_element(By.TAG_NAME, "body").send_keys(Keys.CONTROL + Keys.HOME)
        time.sleep(1)
        print(k)
        rs = download(k)
        if rs:
            with open("done_1.json", "r") as f:
                done = json.load(f)

            done.append(k)

            with open("done_1.json", "w") as f:
                json.dump(done, f)

run(1, 150)